
<h1 style="text-align: center;">
Implied Volatility Computation using Bisection and Newton-Raphson Methods
</h1>

## 0. The Black-Scholes model

The Black-Scholes model provides a closed-form formula for pricing European options.  Most of its components are directly observed in the market; however, volatility is not quoted explicitly.

The model states that the price of a European call option (assuming the underlying pays a continuous dividend yield; set $q=0$ if there are no dividends) is given by:

$$
C = S_0 e^{-qT}\,\Phi(d_1) - K e^{-rT}\,\Phi(d_2)
$$

where

$$
d_1 = \frac{\ln\!\left(\frac{S_0}{K}\right) + \left(r - q + \tfrac{1}{2}\sigma^2\right)T}{\sigma\sqrt{T}},
\qquad
d_2 = d_1 - \sigma\sqrt{T}.
$$

---

## 1. Implied Volatility (IV)

Implied volatility refers to the level of volatility that makes the Black–Scholes model price match the observed market price. It is called *implied* because it is inferred by inverting the pricing model rather than being directly observed.

However, the Black–Scholes formula cannot be algebraically inverted to express volatility as a closed-form function of the option price. There is no explicit analytical formula for implied volatility.

#### 1.1 How to compute (IV)?

Implied volatility is computed using numerical root-finding methods. We define the target function:

$$
f(\sigma) = BS(\sigma) - C_{\text{mkt}},
$$

where $BS(\sigma)$ is the Black–Scholes price evaluated at volatility $\sigma$, and $C_{\text{mkt}}$ is the observed market price.

If$$
f(\sigma^*) = 0,
$$

then the model price matches the market price.

---

## 2. Numerical Methods (1): Bisection Method


The bisection method is a simple and robust root-finding algorithm.

**Procedure:**

1. Define an interval $[\sigma_L,\sigma_U]$.
2. Split the interval in half.
3. Evaluate the function at the midpoint.
4. Keep the subinterval that contains the sign change.
5. Repeat until the interval is sufficiently small.

####  2.1 Intermediate Value Theorem

Let $f$ be continuous on $[\sigma_L, \sigma_U]$.

If$$
f(\sigma_L)\, f(\sigma_U) < 0,
$$

then there exists at least one $\sigma^* \in (\sigma_L, \sigma_U)$ such that

$$
f(\sigma^*) = 0.
$$

**Interpretation:**

- A sign change guarantees a root.
- If both endpoints have the same sign, the interval does not contain a root.

Recall that:

$$
f(\sigma) = BS(\sigma) - C_{\text{mkt}}.
$$

If both $f(\sigma_L)$ and $f(\sigma_U)$ are negative:

$$
BS(\sigma) < C_{\text{mkt}}.
$$

Since the Black–Scholes price is strictly increasing in volatility (Vega > 0),
no value inside the interval can match the market price.

####  2.2 Interval Splitting

Start with:

$$
[\sigma_L, \sigma_U], \quad f(\sigma_L)\, f(\sigma_U) < 0.
$$

Compute midpoint:

$$
\sigma_M^{(1)} = \frac{\sigma_L + \sigma_U}{2}.
$$

Split into:

$$
[\sigma_L, \sigma_M^{(1)}]
\quad \text{and} \quad
[\sigma_M^{(1)}, \sigma_U].
$$

After $n$ iterations:

$$
\sigma_U^{(n)} - \sigma_L^{(n)} = 
\frac{\sigma_U^{(0)} - \sigma_L^{(0)}}{2^n}.
$$

As $n \to \infty$:

$$
\sigma_U^{(n)} - \sigma_L^{(n)} \to 0.
$$

In practice, we stop when:

$$
\sigma_U^{(n)} - \sigma_L^{(n)} < \varepsilon.
$$

---
## 3. Practical Caveat: Bracketing May Fail

In practice, the initial interval

$$
[\sigma_{\text{low}}, \sigma_{\text{high}}]
$$

may not satisfy

$$
f(\sigma_{\text{low}})\, f(\sigma_{\text{high}}) < 0.
$$

####  3.1 Implementation Strategy

- Start with a very small lower bound (e.g. $\sigma_{\text{low}} = 10^{-8}$).
- Choose an initial upper bound (e.g. $\sigma_{\text{high}} = 2.0$).
- If the sign condition fails, repeatedly expand the upper bound:

$$
\sigma_{\text{high}} \leftarrow 2\, \sigma_{\text{high}}.
$$

- Stop after a maximum number of expansions (e.g. 10).

If the sign condition is still not satisfied, the algorithm returns failure.

#### 3.2 Why Expand the Upper Bound?


If

$$
f(\sigma_{\text{low}})\, f(\sigma_{\text{high}}) > 0,
$$

both endpoints imply the same pricing inequality.

Since the Black–Scholes price is increasing in volatility,
a higher volatility may be required to reach the market price.

After at most 10 expansions:

$$
\sigma_{\text{high}} = 2^{10} \times 2.0 \approx 2000\%.
$$

If no sign change occurs, the implied volatility is likely unrealistic,
and the routine stops.

#### 3.3 Arbitrage Lower Bound

Before running the algorithm, we must verify the no-arbitrage condition:

$$
C_{\text{mkt}} \ge 
\max\!\left( S_0 e^{-qT} - K e^{-rT}, \; 0 \right).
$$

If this condition fails, no implied volatility exists.

In [102]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt

#Theoretical price of a call option under Black-Scholes

def bs_call_price(S0, K, r, q, T, sigma):
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)


def implied_vol_bisection_bracket(price_mkt, S0, K, r, q, T,
                                  sigma_low=1e-8, sigma_high=2.0,
                                  tol=1e-6, max_iter=100, expand_max=10):
    """
    Implied volatility via bisection with explicit bracketing:
      requires f(low)*f(high) < 0, where f(sigma)=BS(sigma)-price_mkt.
      ---------------------------------
      @ time t
      price_mkt:   Observed market's price 
      S0:          Underlying's spot price
      K:           Option's strike price
      r:           Interest rate used
      q:           Annual Dividend rate paid by the underlying (if not set to 0)
      T:           Option's time to maturity
      ---------------------------------
      sigma_low:   First lower bound guess
      sigma_high:  First upper bound guess
      tol:         Interval Length of the interval resulting from each iteration (a really small intervla means that
                   the process converged to the adequate IV.
      max_iter:    Maximum number of times in which we split intervals
      expand_max:  Maximum number of times in which we expand the upper bracket of the vol interval by 2.0
      ---------------------------------
    """

    # Arbitrage lower bound
    intrinsic = max(S0 * np.exp(-q * T) - K * np.exp(-r * T), 0.0)
    if price_mkt < intrinsic:
        return f"FAIL: price < intrinsic (mid={price_mkt:.2f} < {intrinsic:.2f})"


    # Define f(sigma)
    def f(sigma):
        return bs_call_price(S0, K, r, q, T, sigma) - price_mkt

    # Bracket the root: f(low)*f(high) < 0
    f_low = f(sigma_low)
    f_high = f(sigma_high)

    #Keep expanding the upper bound as f_low * f_high > 0 for every iteration i
    j = 0
    while (f_low * f_high > 0) and (j < expand_max):
        sigma_high *= 2.0
        f_high = f(sigma_high)
        j += 1

    # If still no bracket after expand_max of iterations, give up
    if f_low * f_high > 0:
        return f"FAIL: could not bracket after {j} expansions. Vol is over: {sigma_high:.2f}"
        

    # Everytime f_low * f_high < 0, we define a new bracket
    low, high = sigma_low, sigma_high
    i = 0

    while (high - low > tol) and (i < max_iter):
        mid = 0.5 * (low + high)
        f_mid = f(mid)

        # Keep the interval that preserves sign change
        if f_low * f_mid <= 0:
            high = mid
            f_high = f_mid

         # if f_low * f_mid > 0:
        else:
            low = mid
            f_low = f_mid

        i += 1

    return 0.5 * (low + high)


In [77]:
#IV computation using Bisection-Method

T  = 2.841096
S0 = 691.960022
q  = 0.013363	
r  = 0.036109
K  = 360
price_mkt = 362.250

print("The implied volatility estimate under the Bisection Method is: ", 
      implied_vol_bisection_bracket(price_mkt = price_mkt, S0 = S0, K = K, r = r, q = q, T = T,
                                  sigma_low=1e-8, sigma_high=2.0,
                                  tol=1e-6, max_iter=100, expand_max=10))

The implied volatility estimate under the Bisection Method is:  0.3924298366816497


In [84]:
#Let's save estimate in the object sigma and then use it to compute the BS price,
#which should math the option's market price
sigma =implied_vol_bisection_bracket(price_mkt = 362.250, S0 = S0, K = K, r = r, q = q, T = T,
                                  sigma_low=1e-8, sigma_high=2.0,
                                  tol=1e-6, max_iter=100, expand_max=10)

print("The difference between the market price and the BS theoretical price is:",
      price_mkt  - bs_call_price(S0, K, r, q, T, sigma))

The difference between the market price and the BS theoretical price is: 4.5958665907619434e-05




## 4. Numerical Methods (2): Newton–Raphson Method

#### 4.1 Problem Setup

We want to solve the implied volatility equation:

$$
f(\sigma) = BS(\sigma) - C_{\text{mkt}} = 0.
$$

That is, we seek $\sigma^*$ such that

$$
BS(\sigma^*) = C_{\text{mkt}}.
$$

Unlike the bisection method, which shrinks a bracketing interval, Newton–Raphson updates a single estimate using local slope information.  
This typically leads to much faster convergence when the initial guess is reasonable.


#### 4.2. High-Level Contrast with Bisection

Both methods solve

$$
f(\sigma) = BS(\sigma) - C_{\text{mkt}} = 0,
$$

but they behave differently:

- **Bisection:** requires a bracket $[\sigma_L,\sigma_U]$ with  
  $f(\sigma_L)f(\sigma_U)<0$ and halves the interval each step.
- **Newton–Raphson:** starts from a guess $\sigma^{(0)}$ and uses derivative information to jump toward the root.

**Trade-off:**  
Bisection is robust (guaranteed convergence if bracketed), while Newton–Raphson is faster but may fail with a poor initial guess or small slope.


#### 4.3. Starting from a Guess

Newton–Raphson begins with an initial guess:

$$
\sigma^{(0)}.
$$

At iteration $k$, we have the estimate:

$$
\sigma^{(k)}.
$$

We approximate the function locally around $\sigma^{(k)}$.


#### 4.4. Local Linear Approximation

Using a first-order Taylor expansion around $\sigma^{(k)}$:

$$
f(\sigma)
\approx
f(\sigma^{(k)})
+
f'(\sigma^{(k)})(\sigma - \sigma^{(k)}).
$$

This is the tangent-line approximation of $f$.

Imposing the root condition:

$$
f(\sigma) = 0.
$$


#### 4.5. Deriving the Update Rule

Substitute into the approximation:

$$
0
=
f(\sigma^{(k)})
+
f'(\sigma^{(k)})(\sigma - \sigma^{(k)}).
$$

Rearranging:

$$
\sigma - \sigma^{(k)}
=
-
\frac{f(\sigma^{(k)})}{f'(\sigma^{(k)})}.
$$

Therefore,

$$
\boxed{
\sigma^{(k+1)}
=
\sigma^{(k)}
-
\frac{f(\sigma^{(k)})}{f'(\sigma^{(k)})}
}
$$

This is the Newton–Raphson update rule.


#### 4.6 This can be rewritten as:

Recall:

$$
f(\sigma) = BS(\sigma) - C_{\text{mkt}}.
$$

Then

$$
f'(\sigma)
=
\frac{\partial BS(\sigma)}{\partial \sigma}
=
\text{Vega}(\sigma).
$$

where $$
\text{Vega}(\sigma)
=
\frac{\partial C}{\partial \sigma}
=
S_0 e^{-qT} \phi(d_1)\sqrt{T}
$$


The practical update becomes:

$$
\boxed{
\sigma^{(k+1)}
=
\sigma^{(k)}
-
\frac{BS(\sigma^{(k)}) - C_{\text{mkt}}}
{\text{Vega}(\sigma^{(k)})}
}
$$

---

## 5. Interpretation of the Newton Step

We can write:

$$
\sigma^{(k+1)}
=
\sigma^{(k)}
-
\frac{\text{Pricing error}}
{\text{Sensitivity (Vega)}}.
$$

**Pricing error:**

$$
f(\sigma^{(k)}) = BS(\sigma^{(k)}) - C_{\text{mkt}}.
$$

**Sensitivity:**

$$
f'(\sigma^{(k)}) = \text{Vega}(\sigma^{(k)}).
$$

### Intuition

- If Vega is large → small volatility adjustment.
- If Vega is small → larger volatility adjustment.

We scale the pricing error by the local slope to determine the appropriate correction.

---

## 6. Convergence Behavior

As iterations progress:

$$
f(\sigma^{(k)}) \to 0,
$$

$$
\frac{f(\sigma^{(k)})}{\text{Vega}(\sigma^{(k)})} \to 0,
$$

$$
\sigma^{(k+1)} - \sigma^{(k)} \to 0.
$$

Near the solution, convergence is **quadratic**:

$$
|\sigma^{(k+1)} - \sigma^*|
\approx
C\,|\sigma^{(k)} - \sigma^*|^2.
$$

This means the number of correct digits roughly doubles at each step.

---

## 7. Important Caveat

If

$$
\text{Vega}(\sigma^{(k)}) \approx 0,
$$

then

$$
\frac{f(\sigma^{(k)})}{\text{Vega}(\sigma^{(k)})}
$$

can become very large, leading to unstable jumps.

Newton–Raphson is therefore **fast but not always robust**.

In [107]:
def bs_call_vega(S0, K, r, q, T, sigma):
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S0 * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)

def implied_vol_newton_or_bisect(price_mkt, S0, K, r, q, T,
                                 sigma0=0.2, tol=1e-6, max_iter=100,
                                 vega_min=1e-8):
    """
    Implied volatility via Newton–Raphson.
    Falls back to bisection with explicit bracketing if Vega is very small.

    f(sigma) = BS(sigma) - price_mkt
      ---------------------------------
      @ time t
      price_mkt:   Observed market's price 
      S0:          Underlying's spot price
      K:           Option's strike price
      r:           Interest rate used
      q:           Annual Dividend rate paid by the underlying (if not set to 0)
      T:           Option's time to maturity
      ---------------------------------
      sigma0:      First sigma guess
      tol:         Acceptable pricing error
      max_iter:    Maximum number of times in which we adjust sigma by the correction term (pricing error/vega)
      vega_min:    Smallest acceptable value for vega to achieve convergence
      ---------------------------------
    """
    
    sigma = float(sigma0)

    for _ in range(max_iter):
        f_val = bs_call_price(S0, K, r, q, T, sigma) - price_mkt #pricing error

        if abs(f_val) < tol:
            return sigma

        vega = bs_call_vega(S0, K, r, q, T, sigma)

        if abs(vega) < vega_min:
            # fallback: bisection is robust when vega is tiny
            return implied_vol_bisection_bracket(price_mkt, S0, K, r, q, T)

        sigma = sigma - f_val / vega

        # keep sigma in a valid range
        if sigma <= 0:
            sigma = 1e-8

    return np.nan

In [112]:
T  = 2.841096
S0 = 691.960022
q  = 0.013363	
r  = 0.036109
K  = 360
price_mkt = 362.250

print("The implied volatility estimate under the Bisection Method is: ", 
      implied_vol_bisection_bracket(price_mkt = price_mkt, S0 = S0, K = K, r = r, q = q, T = T,
                                  sigma_low=1e-8, sigma_high=2.0,
                                  tol=1e-6, max_iter=100, expand_max=10))

print( "The implied volatility estimate under the Newton-Raphson is: " ,implied_vol_newton_or_bisect(price_mkt= price_mkt, S0 = S0, K = K, r = r, q = q, T = T,
                                 sigma0=0.2, tol=1e-6, max_iter=100,
                                 vega_min=1e-8))
#NR is faster
%timeit implied_vol_newton_or_bisect(price_mkt= price_mkt, S0 = S0, K = K, r = r, q = q, T = T, sigma0=0.2, tol=1e-6, max_iter=100, vega_min=1e-8)
#Bisection
%timeit implied_vol_bisection_bracket(price_mkt = price_mkt, S0 = S0, K = K, r = r, q = q, T = T, sigma_low=1e-8, sigma_high=2.0,tol=1e-6, max_iter=100, expand_max=10)

The implied volatility estimate under the Bisection Method is:  0.3924298366816497
The implied volatility estimate under the Newton-Raphson is:  0.3924301163764797
517 μs ± 18.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.32 ms ± 124 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
